In [1]:
import pandas as pd
import numpy as np
import torch
import random
data = pd.read_csv('DATA3_filter.csv')

In [2]:
ctrl_o2 = data["3UD5NAO032"] #省煤器出口氧量
ctrl_coal = data[["3U28M04AFT", "3U28M04BFT", "3U28M04CFT", "3U28M04DFT"]] #给煤机给煤量
ctrl_sec = data[["3U71MCV07AK", "3U71MCV07BK", "3U71MCV07CK", "3U71MCV07DK"]] #二次风门开度
ctrl_sofa = data[["3USOFABJ1ZT"]] #燃尽风门开度

In [11]:
state_all = data[["MW", "3UCCSPT", "3U03TEA", "3UZQFT", "3U05TE01", "3U06PT04", "3U80FT04", "3U80FT05", "3UTAFF", "3U72PT04", "3UBCFHHTL"]]  # 状态变量
target_atem = data["3U78TE02A"]  # A排烟温度
target_btem = data["3U78TE02B"]  # B排烟温度
target_co = data["co_filter"]  # CO浓度
target_nox = data["nox_filter"]  # NOx浓度
target_eff = data["eff"]  # 锅炉热效率
load = data["MW"]  # 机组负荷

# 变量的描述信息

In [12]:
def print_distribution(data, points=[0.1,1,2,3,97,98,99,99.9]):
    for point in points:
        print(f"分位数{point}:",np.percentile(data, point))
    data_inc = (np.roll(data, -1) - data)[:-1]
    for point in points:
        print(f"增量分位数{point}:",np.percentile(data_inc, point))

print_distribution(np.array(target_nox), points=[0.1,99.9])

分位数0.1: 497.069328
分位数99.9: 3419.5708480000244
增量分位数0.1: -50.0
增量分位数99.9: 50.0


In [17]:
ctrl_o2_limit = [[1.8], [4.8]]
ctrl_coal_limit = [[0, 0, 0, 0, 0], [40, 40, 40, 40, 40]]
ctrl_sec_limit = [[0, 0, 0, 0, 0, 0, 0, 0, 0], [75, 75, 75, 75, 75, 75, 75, 75, 75]]
ctrl_sofa_limit = [[0, 0, 0, 0], [100, 100, 100, 100]]
state_all_limit = [[110, 535, 1.1, 0, 0, 0, 0, 0, 0], [330, 550, 3.4, 2, 55, 55, 55, 55, 55]]
target_atem_limit = [[117], [157]]
target_co_limit = [[20], [1500]]
target_nox_limit = [[130], [280]]
target_btem_limit = [[517], [547]]

In [20]:
# 确保 reshape 正确
def shape2d(data):
    data = np.array(data)
    if len(data.shape) == 1:
        return data.reshape(-1, 1)
    return data

In [21]:
# 归一化函数
def minMaxNormalize(data, min_value, max_value):
    return (data - min_value) / (max_value - min_value)

In [22]:
# 准备数据
data_ctrl = np.concatenate([shape2d(ctrl_o2), shape2d(ctrl_coal), shape2d(ctrl_sec), shape2d(ctrl_sofa)], axis=1)
data_state = shape2d(state_all)
data_target = np.concatenate([shape2d(target_atem), shape2d(target_btem), shape2d(target_co), shape2d(target_nox)], axis=1)

In [23]:
data_ctrl_limit = np.concatenate([shape2d(ctrl_o2_limit), shape2d(ctrl_coal_limit), shape2d(ctrl_sec_limit), shape2d(ctrl_sofa_limit)], axis=1)
data_state_limit = shape2d(state_all_limit)
data_target_limit = np.concatenate([shape2d(target_atem_limit), shape2d(target_btem_limit), shape2d(target_co_limit), shape2d(target_nox_limit)], axis=1)

In [24]:
# 打印形状以进行调试
print("data_ctrl 的形状:", data_ctrl.shape)
print("data_ctrl_limit 的形状:", data_ctrl_limit.shape)
print("data_state 的形状:", data_state.shape)
print("data_state_limit 的形状:", data_state_limit.shape)
print("data_target 的形状:", data_target.shape)
print("data_target_limit 的形状:", data_target_limit.shape)

data_ctrl 的形状: (133921, 10)
data_ctrl_limit 的形状: (2, 19)
data_state 的形状: (133921, 11)
data_state_limit 的形状: (2, 9)
data_target 的形状: (133921, 4)
data_target_limit 的形状: (2, 4)


In [25]:
# 检查形状是否匹配
assert data_ctrl.shape[1] == data_ctrl_limit.shape[1], "控制数据和限制特征数量不匹配"
assert data_state.shape[1] == data_state_limit.shape[1], "状态数据和限制特征数量不匹配"
assert data_target.shape[1] == data_target_limit.shape[1], "目标数据和限制特征数量不匹配"

AssertionError: 控制数据和限制特征数量不匹配